# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    dfTeams = nhl_df[nhl_df['year']==2018][['team','W','L']]
    dfTeams.drop([0,9,18,26],inplace=True)
    
    dfTeams['team'] = dfTeams['team'].replace("\*", "", regex=True)
    dfTeams['team'] = dfTeams['team'].str.split(' ').apply(lambda x: x[-1])
    dfTeams = dfTeams.astype({'team': str,'W': int, 'L': int})
    dfTeams['W/L'] =  dfTeams['W']/(dfTeams['W']+dfTeams['L'])
    
    
    dfcities = cities[['Metropolitan area','Population (2016 est.)[8]','NHL']]
    dfcities['NHL'].replace("\[.*\]", "", regex=True,inplace=True)
    dfcities['NHL'].replace("—", np.nan, regex=True,inplace=True)
    dfcities['NHL'].replace("", np.nan, regex=True,inplace=True)
    dfcities['team'] = dfcities['NHL'].str.findall('([A-Z0-9][^A-Z]*)')
    dfcities = dfcities.assign(team=dfcities['team']).explode('team')
    dfcities.reset_index(inplace=True)
    del(dfcities['index'],dfcities['NHL'])

    display(dfcities)
#     display(dfTeams)
    
    df = pd.merge(dfcities,dfTeams, how='inner',on='team')
    df.rename(columns={'Population (2016 est.)[8]':'population'},inplace=True)
    
    df = df.astype({'team': str, 'population':int})
    
    df = df.groupby('Metropolitan area').agg({"W/L":np.nanmean,"population":np.nanmean})
#     display(df)
#     raise NotImplementedError()
    
    population_by_region = df['population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nhl_correlation()

,Metropolitan area,Population (2016 est.)[8],team
0,New York City,20153634,Rangers
1,New York City,20153634,Islanders
2,New York City,20153634,Devils
3,Los Angeles,13310447,Kings
4,Los Angeles,13310447,Ducks
5,San Francisco Bay Area,6657982,Sharks
6,Chicago,9512999,Blackhawks
7,Dallas–Fort Worth,7233323,Stars
8,"Washington, D.C.",6131977,Capitals
9,Philadelphia,6070500,Flyers


0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [55]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    dfTeams = nba_df[nba_df['year']==2018][['team','W/L%']]
#     display(dfTeams)    
    
    dfTeams['team'].replace("\(.*\)", "", regex=True,inplace=True)
    dfTeams['team'] = dfTeams['team'].replace("\*", "", regex=True)
    dfTeams['team'] = dfTeams['team'].str.split(' ').apply(lambda x: x[-1])
    dfTeams['team'] = dfTeams['team'].str.strip()
    dfTeams = dfTeams.astype({'team':str})
#     display(dfTeams)
    
    dfcities = cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]
    dfcities['NBA'].replace("\[.*\]", "", regex=True,inplace=True)
    dfcities['NBA'].replace("—", np.nan, regex=True,inplace=True)
    dfcities['NBA'].replace("", np.nan, regex=True,inplace=True)
    dfcities['team'] = dfcities['NBA'].str.findall('([A-Z0-9][^A-Z]*)')
    dfcities = dfcities.assign(team=dfcities['team']).explode('team')
    dfcities.reset_index(inplace=True)
    del(dfcities['index'],dfcities['NBA'])
    dfcities = dfcities.astype({'team':str})
#     display(dfcities)

    df = pd.merge(dfcities,dfTeams, how='inner',on='team')
    df.rename(columns={'Population (2016 est.)[8]':'population'},inplace=True)
    
    df = df.astype({'W/L%': float, 'population':int})
    
    df = df.groupby('Metropolitan area').agg({"W/L%":np.nanmean,"population":np.nanmean})
#     display(df)
    
#     raise NotImplementedError()
    
    population_by_region = df['population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
# nba_correlation()

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [56]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    dfTeams = mlb_df[mlb_df['year']==2018][['team','W-L%']]
#     display(dfTeams)    
    
    dfTeams['team'] = dfTeams['team'].str.strip()
    dfTeams['team'].replace({"Boston Red Sox":"Boston Redsox",
                              "Chicago White Sox": "Chicago Whitesox",
                              "Toronto Blue Jays":"Toronto Bluejays"},inplace=True)
    dfTeams['team'] = dfTeams['team'].str.split(' ').apply(lambda x: x[-1])
    dfTeams = dfTeams.astype({'team':str})
#     display(dfTeams)
    
    dfcities = cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]
    dfcities['MLB'].replace("\[.*\]", "", regex=True,inplace=True)
    dfcities['MLB'].replace("—", np.nan, regex=True,inplace=True)
    dfcities['MLB'].replace("", np.nan, regex=True,inplace=True)
    dfcities['MLB'].replace({"Red Sox":"Redsox",
                              "CubsWhite Sox": "CubsWhitesox",
                              "Blue Jays":"Bluejays"},inplace=True)
    dfcities['team'] = dfcities['MLB'].str.findall('([A-Z0-9][^A-Z]*)')
    dfcities = dfcities.assign(team=dfcities['team']).explode('team')
    dfcities.reset_index(inplace=True)
    del(dfcities['index'],dfcities['MLB'])
    dfcities = dfcities.astype({'team':str})
#     display(dfcities)

    df = pd.merge(dfcities,dfTeams, how='inner',on='team')
    df.rename(columns={'Population (2016 est.)[8]':'population'},inplace=True)
    
    df = df.astype({'W-L%': float, 'population':int})
    
    df = df.groupby('Metropolitan area').agg({"W-L%":np.nanmean,"population":np.nanmean})
#     display(df)

#     raise NotImplementedError()
    
    population_by_region = df['population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
# mlb_correlation()

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [57]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# display(cities)

def nfl_correlation(): 
    # YOUR CODE HERE
    dfTeams = nfl_df[nfl_df['year']==2018][['team','W-L%']]
#     del(dfTeams['year','League','DSRS','MoV','OSRS','PA','PD','PF','SRS','SoS','T'])
    
    dfTeams.drop([0,5,10,15,20,25,30,35],inplace=True)
    
    dfTeams['team'] = dfTeams['team'].replace("\*", "", regex=True)
    dfTeams['team'] = dfTeams['team'].replace("\+", "", regex=True)
#     df.drop( df[ df['Sex'] == 1 ].index, inplace=True)    
#     dfTeams[dfTeams['L'] in ['AFC East','AFC West','AFC North','AFC South','NFC East','NFC West','NFC North','NFC South']]
#     dfTeams.drop(dfTeams[dfTeams['L'] in ['AFC East','AFC West','AFC North','AFC South','NFC East','NFC West','NFC North','NFC South']].index, inplace = True)
    dfTeams['team'] = dfTeams['team'].replace("\+", "", regex=True)
    dfTeams['team'] = dfTeams['team'].str.split(' ').apply(lambda x: x[-1])
#     team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    
#     display(df)
    dfcities = cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]
    dfcities['NFL'].replace("\[.*\]", "", regex=True,inplace=True)
    dfcities['NFL'].replace("—", np.nan, regex=True,inplace=True)
    dfcities['NFL'].replace("", np.nan, regex=True,inplace=True)
    dfcities['team'] = dfcities['NFL'].str.findall('([A-Z0-9][^A-Z]*)')
    dfcities = dfcities.assign(team=dfcities['team']).explode('team')
    dfcities.reset_index(inplace=True)
    del(dfcities['index'],dfcities['NFL'])
#     display(dfcities)
#     display(dfTeams)
    df = pd.merge(dfcities,dfTeams, how='inner',on='team')
    df.rename(columns={'Population (2016 est.)[8]':'population'},inplace=True)
    
    df = df.astype({'team': str,'W-L%': float, 'population':int})
    
    df = df.groupby('Metropolitan area').agg({"W-L%":np.nanmean,"population":np.nanmean})
#     display(df)
        
#     display(dfcities['NFL'].str.extract('([A-Z0-9][a-z0-9]*|[A-Z0-9][a-z0-9]*)'))
#     display(dfcities)
    population_by_region = df['population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
#     raise NotImplementedError()
    

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
# nfl_correlation()

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# mlb_df=pd.read_csv("assets/mlb.csv")
# nhl_df=pd.read_csv("assets/nhl.csv")
# nba_df=pd.read_csv("assets/nba.csv")
# nfl_df=pd.read_csv("assets/nfl.csv")
# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl(): 
    # YOUR CODE HERE
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    dfTeams = nhl_df[nhl_df['year']==2018][['team','W','L']]
    dfTeams.drop([0,9,18,26],inplace=True)
    
    dfTeams['team'] = dfTeams['team'].replace("\*", "", regex=True)
    dfTeams['team'] = dfTeams['team'].str.split(' ').apply(lambda x: x[-1])
    dfTeams = dfTeams.astype({'team': str,'W': int, 'L': int})
    dfTeams['W/L'] =  dfTeams['W']/(dfTeams['W']+dfTeams['L'])
    
    
    dfcities = cities[['Metropolitan area','Population (2016 est.)[8]','NHL']]
    dfcities['NHL'].replace("\[.*\]", "", regex=True,inplace=True)
    dfcities['NHL'].replace("—", np.nan, regex=True,inplace=True)
    dfcities['NHL'].replace("", np.nan, regex=True,inplace=True)
    dfcities['team'] = dfcities['NHL'].str.findall('([A-Z0-9][^A-Z]*)')
    dfcities = dfcities.assign(team=dfcities['team']).explode('team')
    dfcities.reset_index(inplace=True)
    del(dfcities['index'],dfcities['NHL'])

#     display(dfcities)
#     display(dfTeams)
    
    df = pd.merge(dfcities,dfTeams, how='inner',on='team')
    df.rename(columns={'Population (2016 est.)[8]':'population'},inplace=True)
    
    df = df.astype({'team': str, 'population':int})
    
    df = df.groupby('Metropolitan area').agg({"W/L":np.nanmean,"population":np.nanmean})
    df.rename(columns={'W/L':'W-L%NHL','team':'teamNHL'},inplace=True)
#     display(df)
    
    return df

def nba():
    # YOUR CODE HERE
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    dfTeams = nba_df[nba_df['year']==2018][['team','W/L%']]
#     display(dfTeams)    
    
    dfTeams['team'].replace("\(.*\)", "", regex=True,inplace=True)
    dfTeams['team'] = dfTeams['team'].replace("\*", "", regex=True)
    dfTeams['team'] = dfTeams['team'].str.split(' ').apply(lambda x: x[-1])
    dfTeams['team'] = dfTeams['team'].str.strip()
    dfTeams = dfTeams.astype({'team':str})
#     display(dfTeams)
    
    dfcities = cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]
    dfcities['NBA'].replace("\[.*\]", "", regex=True,inplace=True)
    dfcities['NBA'].replace("—", np.nan, regex=True,inplace=True)
    dfcities['NBA'].replace("", np.nan, regex=True,inplace=True)
    dfcities['team'] = dfcities['NBA'].str.findall('([A-Z0-9][^A-Z]*)')
    dfcities = dfcities.assign(team=dfcities['team']).explode('team')
    dfcities.reset_index(inplace=True)
    del(dfcities['index'],dfcities['NBA'])
    dfcities = dfcities.astype({'team':str})
#     display(dfcities)

    df = pd.merge(dfcities,dfTeams, how='inner',on='team')
    df.rename(columns={'Population (2016 est.)[8]':'population'},inplace=True)
    
    df = df.astype({'W/L%': float, 'population':int})
    
    df = df.groupby('Metropolitan area').agg({"W/L%":np.nanmean,"population":np.nanmean})
    df.rename(columns={'W/L%':'W-L%NBA','team':'teamNBA'},inplace=True)
#     display(df)

    return df

def mlb(): 
    # YOUR CODE HERE
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    dfTeams = mlb_df[mlb_df['year']==2018][['team','W-L%']]
#     display(dfTeams)    
    
    dfTeams['team'] = dfTeams['team'].str.strip()
    dfTeams['team'].replace({"Boston Red Sox":"Boston Redsox",
                              "Chicago White Sox": "Chicago Whitesox",
                              "Toronto Blue Jays":"Toronto Bluejays"},inplace=True)
    dfTeams['team'] = dfTeams['team'].str.split(' ').apply(lambda x: x[-1])
    dfTeams = dfTeams.astype({'team':str})
#     display(dfTeams)
    
    dfcities = cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]
    dfcities['MLB'].replace("\[.*\]", "", regex=True,inplace=True)
    dfcities['MLB'].replace("—", np.nan, regex=True,inplace=True)
    dfcities['MLB'].replace("", np.nan, regex=True,inplace=True)
    dfcities['MLB'].replace({"Red Sox":"Redsox",
                              "CubsWhite Sox": "CubsWhitesox",
                              "Blue Jays":"Bluejays"},inplace=True)
    dfcities['team'] = dfcities['MLB'].str.findall('([A-Z0-9][^A-Z]*)')
    dfcities = dfcities.assign(team=dfcities['team']).explode('team')
    dfcities.reset_index(inplace=True)
    del(dfcities['index'],dfcities['MLB'])
    dfcities = dfcities.astype({'team':str})
#     display(dfcities)

    df = pd.merge(dfcities,dfTeams, how='inner',on='team')
    df.rename(columns={'Population (2016 est.)[8]':'population'},inplace=True)
    
    df = df.astype({'W-L%': float, 'population':int})
    
    df = df.groupby('Metropolitan area').agg({"W-L%":np.nanmean,"population":np.nanmean})
    df.rename(columns={'W-L%':'W-L%MLB','team':'teamMLB'},inplace=True)
#     display(df)

    return df

def nfl(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    dfTeams = nfl_df[nfl_df['year']==2018][['team','W-L%']]
#     del(dfTeams['year','League','DSRS','MoV','OSRS','PA','PD','PF','SRS','SoS','T'])
    
    dfTeams.drop([0,5,10,15,20,25,30,35],inplace=True)
    
    dfTeams['team'] = dfTeams['team'].replace("\*", "", regex=True)
    dfTeams['team'] = dfTeams['team'].replace("\+", "", regex=True)
#     df.drop( df[ df['Sex'] == 1 ].index, inplace=True)    
#     dfTeams[dfTeams['L'] in ['AFC East','AFC West','AFC North','AFC South','NFC East','NFC West','NFC North','NFC South']]
#     dfTeams.drop(dfTeams[dfTeams['L'] in ['AFC East','AFC West','AFC North','AFC South','NFC East','NFC West','NFC North','NFC South']].index, inplace = True)
    dfTeams['team'] = dfTeams['team'].replace("\+", "", regex=True)
    dfTeams['team'] = dfTeams['team'].str.split(' ').apply(lambda x: x[-1])
#     team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    
#     display(df)
    dfcities = cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]
    dfcities['NFL'].replace("\[.*\]", "", regex=True,inplace=True)
    dfcities['NFL'].replace("—", np.nan, regex=True,inplace=True)
    dfcities['NFL'].replace("", np.nan, regex=True,inplace=True)
    dfcities['team'] = dfcities['NFL'].str.findall('([A-Z0-9][^A-Z]*)')
    dfcities = dfcities.assign(team=dfcities['team']).explode('team')
    dfcities.reset_index(inplace=True)
    del(dfcities['index'],dfcities['NFL'])
#     display(dfcities)
#     display(dfTeams)
    df = pd.merge(dfcities,dfTeams, how='inner',on='team')
    df.rename(columns={'Population (2016 est.)[8]':'population'},inplace=True)
    
    df = df.astype({'team': str,'W-L%': float, 'population':int})
    
    df = df.groupby('Metropolitan area').agg({"W-L%":np.nanmean,"population":np.nanmean})
    df.rename(columns={'W-L%':'W-L%NFL','team':'teamNFL'},inplace=True)
#     display(df)

    return df

def sports_team_performance():
    # YOUR CODE HERE
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    dfFinal = pd.DataFrame(columns=sports,index=sports)
    dfNFL = nfl()[['W-L%NFL']]
    dfNBA = nba()[['W-L%NBA']]    
    dfMLB = mlb()[['W-L%MLB']]
    dfNHL = nhl()[['W-L%NHL']]
#     display(dfFinal)
    
#     df1 = pd.merge(dfNFL,dfNBA,how='inner',on ='Metropolitan area')
#     dfFinal.loc['NFL','NBA']=stats.ttest_rel(df1['W-L%NFL'], df1['W-L%NBA'])[1]
#     dfFinal.loc['NBA','NFL']=dfFinal.loc['NFL','NBA']
#     df1 = pd.merge(dfNFL,dfMLB,how='inner',on ='Metropolitan area')
#     dfFinal.loc['NFL','MLB']=stats.ttest_rel(df1['W-L%NFL'], df1['W-L%MLB'])[1]
#     dfFinal.loc['MLB','NFL']=dfFinal.loc['NFL','MLB']
#     df1 = pd.merge(dfNFL,dfNHL,how='inner',on ='Metropolitan area')
#     dfFinal.loc['NFL','NHL']=stats.ttest_rel(df1['W-L%NFL'], df1['W-L%NHL'])[1]
#     dfFinal.loc['NHL','NFL']=dfFinal.loc['NFL','NHL']
#     df1 = pd.merge(dfNBA,dfMLB,how='inner',on ='Metropolitan area')
#     dfFinal.loc['NBA','MLB']=stats.ttest_rel(df1['W-L%NBA'], df1['W-L%MLB'])[1]
#     dfFinal.loc['MLB','NBA']=dfFinal.loc['NBA','MLB']
#     df1 = pd.merge(dfNBA,dfNHL,how='inner',on ='Metropolitan area')
#     dfFinal.loc['NBA','NHL']=stats.ttest_rel(df1['W-L%NBA'], df1['W-L%NHL'])[1]
#     dfFinal.loc['NHL','NBA']=dfFinal.loc['NBA','NHL']
#     df1 = pd.merge(dfMLB,dfNHL,how='inner',on ='Metropolitan area')
#     dfFinal.loc['MLB','NHL']=stats.ttest_rel(df1['W-L%MLB'], df1['W-L%NHL'])[1]
#     dfFinal.loc['NHL','MLB']=dfFinal.loc['MLB','NHL']

    df1 = pd.merge(dfNFL,dfNBA,how='outer',on ='Metropolitan area')
    df1 = pd.merge(df1,dfMLB,how='outer',on ='Metropolitan area')
    df1 = pd.merge(df1,dfNHL,how='outer',on ='Metropolitan area')
    for i in sports:
        for  j in sports[sports.index(i)+1:]:
            dfFinal.loc[i,j]=stats.ttest_rel(df1['W-L%'+i], df1['W-L%'+j],nan_policy='omit')[1]
            dfFinal.loc[j,i]=dfFinal.loc[i,j]
    
    p_values = dfFinal
#     display(dfFinal)
    
#     raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
#     sports = ['NFL', 'NBA', 'NHL', 'MLB']
#     p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
# sports_team_performance()